Импортируем все необходимые библиотеки

In [109]:
pip install nltk natasha scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [566]:
import nltk
import string
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import pymorphy2

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd

In [568]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Выделение обозначения стоп-слов

In [571]:
stop_words = set(stopwords.words('russian'))

In [573]:
texts = [
    "Мама мыла раму и пела песню.",
    "Сегодня прекрасная погода, и мы пошли гулять в парк.",
    "Студенты изучают программирование на Python и создают интересные проекты."
]

Функция лемматизации текста

In [576]:
def lemm(text):
    
    segmenter = Segmenter()
    Vocab = MorphVocab()
    Embeddings = NewsEmbedding()
    Tagger = NewsMorphTagger(Embeddings)
    Stemmer = SnowballStemmer("russian")
    morph = pymorphy2.MorphAnalyzer()

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(Tagger)
    
    lems = []
    for token in doc.tokens:
        if token.lemma and token.lemma.isalnum():  
            lems.append(token.lemma)
        else:
            parsed = morph.parse(token.text)[0]
            if parsed.normal_form.isalnum():
                lems.append(parsed.normal_form)
    
    return lems

Функция токенизации текста(без стоп-слов)

In [579]:
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum()]
    return tokens

Функция удаления стоп-слов(по аналогии с токенизацией, но отдельная функция):

In [582]:
def remove_stop_words(texts):
    def clean_sentence(sentence):
        tokens = word_tokenize(sentence.lower())
        tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
        return tokens
    
    if isinstance(texts, list):  
        if all(isinstance(item, str) for item in texts):  
            return [clean_sentence(sentence) for sentence in texts]
        elif all(isinstance(item, list) for item in texts):  
            return [[word for word in sentence if word not in stop_words] for sentence in texts]
    
    return []

исполнение всех функций блоками:

In [585]:
tokens = [tokenize_text(text) for text in texts] #токенизация

In [587]:
lemms = [lemm(text) for text in texts] #Лемматизация

In [588]:
stopwords = remove_stop_words(texts) #удаление стоп слов

Вывод результатов:

In [590]:
print("Результаты токенизации(без удаления стоп-слов):")
print(tokens)

print("Результаты удаления стоп-слов в исходном тексте:")
print(stopwords)

print("Результаты ламматизации:")
print(lemms)

Результаты токенизации(без удаления стоп-слов):
[['мама', 'мыла', 'раму', 'и', 'пела', 'песню'], ['сегодня', 'прекрасная', 'погода', 'и', 'мы', 'пошли', 'гулять', 'в', 'парк'], ['студенты', 'изучают', 'программирование', 'на', 'python', 'и', 'создают', 'интересные', 'проекты']]
Результаты удаления стоп-слов в исходном тексте:
[['мама', 'мыла', 'раму', 'пела', 'песню'], ['сегодня', 'прекрасная', 'погода', 'пошли', 'гулять', 'парк'], ['студенты', 'изучают', 'программирование', 'python', 'создают', 'интересные', 'проекты']]
Результаты ламматизации:
[['мама', 'мыло', 'рама', 'и', 'петь', 'песня'], ['сегодня', 'прекрасный', 'погода', 'и', 'мы', 'послать', 'гулять', 'в', 'парк'], ['студент', 'изучать', 'программирование', 'на', 'python', 'и', 'создавать', 'интересный', 'проект']]


Создаем список уникальных слов

In [592]:
def build_vocab(tokenized_texts):
    vocab = set()
    for tokens in tokenized_texts:
        vocab.update(tokens)
    return sorted(vocab) 

Строим матрицу

In [594]:
def bag_of_words(tokenized_texts):
    vocab = build_vocab(tokenized_texts)
    word_to_index = {word: i for i, word in enumerate(vocab)}  # Индексы слов
    
    matrix = []
    for tokens in tokenized_texts:
        row = [0] * len(vocab)
        for token in tokens:
            if token in word_to_index:
                row[word_to_index[token]] += 1  # Увеличиваем счётчик слова
        matrix.append(row)
    
    return vocab, matrix

In [595]:
bow_features, bow_matrix = bag_of_words(stopwords)

#красивый вывод на табличке
bow_df = pd.DataFrame(bow_matrix, columns=bow_features)
bow_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [596]:
print("Bag of Words:")
display(bow_df)

Bag of Words:


,python,гулять,изучают,интересные,мама,мыла,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0
Предложение 2,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0
Предложение 3,1,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,1


Вычисляет TF (Term Frequency).

In [598]:
def compute_tf(tokens, vocab):
    word_count = len(tokens)
    tf = {word: tokens.count(word) / word_count for word in vocab}  # Частота слова в документе
    return tf

Вычисляет IDF

In [600]:
def compute_idf(tokenized_texts, vocab):
    num_docs = len(tokenized_texts)
    idf = {}
    for word in vocab:
        doc_count = sum(1 for tokens in tokenized_texts if word in tokens)  # В скольких документах встречается
        idf[word] = math.log((num_docs + 1) / (doc_count + 1)) + 1  # Формула с добавлением 1
    return idf

Строит матрицу

In [611]:
def tf_idf(tokenized_texts):
    vocab = build_vocab(tokenized_texts)
    idf = compute_idf(tokenized_texts, vocab)
    
    tf_idf_matrix = []
    for tokens in tokenized_texts:
        tf = compute_tf(tokens, vocab)
        tf_idf_vector = [tf[word] * idf[word] for word in vocab]  # TF * IDF
        tf_idf_matrix.append(tf_idf_vector)
    
    return vocab, tf_idf_matrix

In [613]:
tfidf_features, tfidf_matrix = tf_idf(stopwords)

#красивый вывод на табличке
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_features)
tfidf_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [615]:
print("\nTF-IDF:")
display(tfidf_df)


TF-IDF:


,python,гулять,изучают,интересные,мама,мыла,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0.000000,0.000000,0.000000,0.000000,0.338629,0.338629,0.000000,0.338629,0.338629,0.000000,0.000000,0.000000,0.000000,0.000000,0.338629,0.000000,0.000000,0.000000
Предложение 2,0.000000,0.282191,0.000000,0.000000,0.000000,0.000000,0.282191,0.000000,0.000000,0.282191,0.282191,0.282191,0.000000,0.000000,0.000000,0.282191,0.000000,0.000000
Предложение 3,0.241878,0.000000,0.241878,0.241878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.241878,0.241878,0.000000,0.000000,0.241878,0.241878
